In [1]:
for ii in 1:4
    addprocs(["heathred"])
end

for ii in 1:3
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(2)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


18-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19

In [2]:
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE
@everywhere using Zlib

using ClusterSoup

In [4]:
using WordEmbeddings
LL,word_indexes, indexed_words = load_word2vec_embeddings("../../Resources/example_code/word2vec/GoogleNews-vectors-negative300.bin", 15000)
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(300,15000)
Dict{String,Int64}
Array{String,1}


In [5]:
training_trees = open("questionbank_sents.jsz","r") do fs
    deserialize(fs)
end;
@pz training_trees

training_trees		Array{Any,1}	(4000,)


In [6]:
r_training_trees = r_chunk_data(training_trees)

18-element Array{RemoteRef,1}:
 RemoteRef(2,1,374) 
 RemoteRef(3,1,376) 
 RemoteRef(4,1,378) 
 RemoteRef(5,1,380) 
 RemoteRef(6,1,382) 
 RemoteRef(7,1,384) 
 RemoteRef(8,1,386) 
 RemoteRef(9,1,388) 
 RemoteRef(10,1,390)
 RemoteRef(11,1,392)
 RemoteRef(12,1,394)
 RemoteRef(13,1,396)
 RemoteRef(14,1,398)
 RemoteRef(15,1,400)
 RemoteRef(16,1,402)
 RemoteRef(17,1,404)
 RemoteRef(18,1,406)
 RemoteRef(19,1,408)

In [7]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e[:] = θ[1: W_e_len]
    rae.b_e[:] = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d[:] = θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len]
    rae.b_d[:] = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end

In [8]:
function loss!(θ::Vector)  
    #warn("loss! not defined")
    grad = similar(θ)
    #loss_and_loss_grad!(θ::Vector)
    cached_loss_and_loss_grad!(θ, grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    cached_loss_and_loss_grad!(θ, grad)
end


rae_outer = RAE(LL,word_indexes,indexed_words,0.001);
@time r_rae_outers = put!(workers(), rae_outer, 2)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    @inbounds for r_rae_outer in r_rae_outers
        update_remote(r_rae_outer, rae->unpack!(rae, θ) )
    end
    
    function get_remote_loss_grad_function(r_rae_outer::RemoteRef)
        @assert r_rae_outer.where == myid()
        rae = fetch(r_rae_outer)        
        function loss_and_loss_grad(tree::(Any,Any))
            Δs, err = UnfoldingRAE.loss_and_loss_grad(rae, tree)
            [pack(Δs...), err]
        end
    end
    loss_and_loss_grads::Array{RemoteRef,1} = map(r_rae_outers) do r_raeouter
        remotecall(r_raeouter.where, get_remote_loss_grad_function, r_raeouter)::RemoteRef
    end
    
    
    ret = prechunked_mapreduce(r_training_trees, loss_and_loss_grads, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

elapsed time: 20.69976938 seconds (751070652 bytes allocated, 2.12% gc time)


loss_and_loss_grad! (generic function with 1 method)

In [9]:
loss_and_loss_grad_cache = Dict{Vector{Float64},(Float64, Vector{Float64})}()
loss_and_loss_grad_cache_hits = 0
loss_and_loss_grad_cache_misses = 0
function cached_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global loss_and_loss_grad_cache
    global loss_and_loss_grad_cache_hits
    global loss_and_loss_grad_cache_misses
    if haskey(loss_and_loss_grad_cache,θ)
        loss_and_loss_grad_cache_hits+=1
        err, grad[:] = loss_and_loss_grad_cache[θ]
        err
    else
        loss_and_loss_grad_cache_misses+=1
        err = loss_and_loss_grad!(θ, grad)
        loss_and_loss_grad_cache[θ] = (err, grad)
        err
    end
end

cached_loss_and_loss_grad! (generic function with 1 method)

In [10]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = cached_loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_LBFGS, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 60)
min_objective!(opt, tracking_loss_and_loss_grad!)

#θ = pack(rae_outer)


(optf,optx,ret) = optimize!(opt,θ)


exception on exception on 8: 16: exception on exception on 13: 9: ERROR: NLopt not properly installed. Please run Pkg.build("NLopt")
 in error at error.jl:21
 in include_string at loading.jl:97
 in include_from_node1 at ./loading.jl:131
 in reload_path at loading.jl:152
 in _require at loading.jl:67
 in anonymous at multi.jl:1284
 in anonymous at multi.jl:848
 in run_work_thunk at multi.jl:621
 in run_work_thunk at multi.jl:630
 in anonymous at task.jl:6
while loading /home/wheel/oxinabox/.julia/v0.3/NLopt/src/NLopt.jl, in expression starting on line 16
ERROR: NLopt not properly installed. Please run Pkg.build("NLopt")
 in error at error.jl:21
 in include_string at loading.jl:97
 in include_from_node1 at ./loading.jl:131
 in reload_path at loading.jl:152
 in _require at loading.jl:67
 in anonymous at multi.jl:1284
 in anonymous at multi.jl:848
 in run_work_thunk at multi.jl:621
 in run_work_thunk at multi.jl:630
 in anonymous at task.jl:6
while loading /home/wheel/oxinabox/.julia/v0.3/

LoadError: θ not defined
while loading In[10], in expression starting on line 21

In [13]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,cached_loss_and_loss_grad!)
#θ = pack(rae_outer)
θ=res.minimum
res = optimize(f, θ, method=:cg, show_trace = true, store_trace = true, iterations = 10);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged                       
@printval res.f_converged 
@printval res.trace
@printval loss_and_loss_grad_cache_hits
@printval loss_and_loss_grad_cache_misses

Iter     Function value   Gradient norm 
     0     2.860171e+00     1.184822e-01
     1     2.845680e+00     3.516121e-02
     2     2.829273e+00     2.479758e-02
     3     2.820921e+00     8.551715e-02
     4     2.820921e+00     5.319460e-02
res.f_calls = 21
res.g_calls = 17
res.iterations = 4
res.f_minimum = 2.8209208456108747
res.gr_converged = false
res.x_converged = false
res.trace = Iter     Function value   Gradient norm 
------   --------------   --------------
     0     2.860171e+00     1.184822e-01
     1     2.845680e+00     3.516121e-02
     2     2.829273e+00     2.479758e-02
     3     2.820921e+00     8.551715e-02
     4     2.820921e+00     5.319460e-02

loss_and_loss_grad_cache_hits = 21
loss_and_loss_grad_cache_misses = 35


In [17]:
open("rae1062.jsz","w") do fs
    serialize(fs, rae_outer)
end;


true

In [16]:
unpack!(rae_outer, res.minimum)


RAE(300x15000 Array{Float32,2}:
 0.0   0.0673199    0.0529562  -0.00851202  …  -0.00431673   -0.0364276 
 0.0  -0.0534466    0.0654598  -0.0342245      -0.0434458    -0.102287  
 0.0   0.0189906    0.0661953   0.0322839      -0.0610469     0.0523496 
 0.0   0.0914278    0.0470722   0.0458679       0.0632749     0.123034  
 0.0   0.0659553    0.0522207  -0.0131429       0.00718527    0.0699603 
 0.0  -0.0836951   -0.0820086  -0.0462207   …   0.0163757     0.015922  
 0.0  -0.00181946  -0.0614145  -0.00094823     -0.00704602   -0.0268985 
 0.0  -0.0250175   -0.11621    -0.0522188       0.0316375     0.00101774
 0.0  -0.0343423    0.0156294   0.0465735      -0.0545858    -0.00612153
 0.0   0.0641359    0.0992929   0.0624509      -0.045451      0.0153189 
 0.0  -0.0609518   -0.0856861  -0.122785    …   0.00710172    0.0174901 
 0.0  -0.036844    -0.028133   -0.0287556      -0.00696247    0.0651355 
 0.0  -0.0450316    0.0522207   0.0515131       0.0614925    -0.00150023
 ⋮                 

In [63]:
importall WordEmbeddings 
function show_best(embedder,ĉ::Embedding, nbest=20, similarity=WordEmbeddings.cosine_sim )
    candidates=WordEmbeddings.neighbour_sims(ĉ,embedder.L, similarity)  
    candidates[find(map(isnan, candidates))]= -Inf #If the zero vector was an option, it will be NaN similar so it is banned from winning
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[embedder.indexed_words[ii] round(score,2)] for (ii,score) in best_cands]...)
end

function show_bests(embedder,ĉs::Embeddings, nbest=20, similarity=WordEmbeddings.cosine_sim)
    hcat([show_best(embedder,ĉs[:,ii],nbest, similarity) for ii in 1:size(ĉs,2)]...)
end


show_bests (generic function with 3 methods)

In [88]:
tr=(("What", ("are", "birds")), "?")
a = fold(rae_outer,tr)
b= unfold(rae_outer, a)
ĉs = hcat(map(leave-> leave.ĉ, b)...)
bests = show_bests(rae_outer, ĉs)

20x8 Array{Any,2}:
 "What"      0.95  "the"       0.7   "just"      0.55  "Following"  0.33
 "How"       0.74  "in"        0.66  "it"        0.52  "In"         0.32
 "Which"     0.72  "one"       0.59  "the"       0.51  "A"          0.32
 "Why"       0.69  "that"      0.58  "that"      0.49  "Next"       0.31
 "Where"     0.69  "just"      0.58  "do"        0.48  "With"       0.31
 "So"        0.68  "where"     0.56  "anyway"    0.48  "On"         0.3 
 "And"       0.65  "only"      0.55  "here"      0.48  "The"        0.3 
 "Whatever"  0.65  "for"       0.55  "one"       0.48  "Is"         0.3 
 "Really"    0.64  "but"       0.54  "but"       0.47  "From"       0.29
 "Actually"  0.64  "this"      0.52  "so"        0.47  "in"         0.29
 "If"        0.63  "actually"  0.52  "he"        0.47  "Taking"     0.29
 "Maybe"     0.62  "What"      0.52  "not"       0.47  "Finally"    0.29
 "Who"       0.61  "when"      0.52  "probably"  0.47  "Consider"   0.29
 "Well"      0.6   "all"       0

In [86]:
haskey(rae_outer.word_index,"?")

false

In [37]:
findfirst(candidates,NaN)

0

In [39]:
select(candidates,1:nbest, rev=true)

3-element Array{Any,1}:
 NaN       
   0.953141
   0.743962

-Inf

In [56]:
candidates

15000-element Array{Any,1}:
 -Inf         
   -0.0133476 
    0.0711397 
    0.112401  
    0.281986  
    0.0200398 
    0.0482833 
    0.0160419 
    0.389349  
    0.0360597 
    0.028397  
   -0.130646  
    0.0152059 
    ⋮         
    0.0338449 
    0.0150838 
    0.0159347 
    0.00824797
    0.0757282 
    0.0207295 
    0.109296  
    0.0335025 
    0.0421385 
    0.0747745 
   -0.00686668
    0.0141052 